## Keras Tutorial

In [ ]:
Docs API: https://www.tensorflow.org/api_docs/python/tf

In [12]:
from keras.datasets.mnist import load_data
import numpy as np

In [2]:
(x_train, y_train), (x_test, y_test) = load_data() # unpacking di python (2 tuple, con due elementi)

In [3]:
# Normalizzazione
x_train = x_test / 255.
x_test = x_test/ 255.
# Perché? load_data() da immagini come valori interi (0-255, in scala di grigi): Normalizzo in range 0-1. 
#         (nota: in questo caso lo sfondo nero va a 0 e questo facilita il lavoro della rete

In [4]:
y_test = y_test.astype(np.int32)
y_train = y_train.astype(np.int32)

In [13]:
import keras
from keras.layers import Dense, Flatten

### Creazione Modello con Sequential

In [9]:
model = keras.Sequential(
    [
        Flatten(),
        Dense(100, "sigmoid"),
        Dense(100, "sigmoid"),
        Dense(10, "softmax")
        #Dense(10) # senza softmax se nel compile() metto from_logits = True
    ]
)

# Nota: dimensioni: (batch size, dimensione dell'input) la Flatten fa una reshape(-1) del tensore in input, e non del batch

### Spiegazione Flattten

In [ ]:
import tensorflow as tf

t = tf.random.normal([32,640, 480, 3])

In [ ]:
t_flatten = tf.reshape(t, shape=[t.shape[0], -1])

In [ ]:
t_flatten.shape

In [2]:
# fine spiegazione

## Continuo Keras

### Model building

In [10]:
model(x_train[:10]) # prende 10 images e da le probabilità (casuali, il modello non è stato allenato, però è BUILT)
# un batch da 10 (10x28x28), potrei anche fare 1x28x28, ma non 0x28x28

# Nota: i modelli si aspettano sempre un batch

<tf.Tensor: shape=(10, 10), dtype=float32, numpy=
array([[0.08239181, 0.10936715, 0.07162292, 0.17838377, 0.02009678,
        0.29507178, 0.01412347, 0.06092006, 0.09651064, 0.07151174],
       [0.08462055, 0.11345805, 0.0715907 , 0.16804957, 0.02008302,
        0.2970242 , 0.01474483, 0.06082911, 0.09899225, 0.07060766],
       [0.08485747, 0.10963368, 0.07436068, 0.17243621, 0.02113033,
        0.29441822, 0.01467785, 0.05945624, 0.09686764, 0.07216157],
       [0.08640997, 0.11222877, 0.07269963, 0.17130844, 0.0206222 ,
        0.28687954, 0.01418974, 0.05762508, 0.10477756, 0.07325892],
       [0.08639479, 0.11159971, 0.07208452, 0.16794686, 0.02098403,
        0.2908852 , 0.01447166, 0.0636696 , 0.09923788, 0.0727258 ],
       [0.08562789, 0.10958152, 0.07496532, 0.17562078, 0.02092306,
        0.28991756, 0.01460755, 0.05911367, 0.09670519, 0.07293756],
       [0.08249418, 0.1104734 , 0.07353803, 0.17621456, 0.0207559 ,
        0.28980252, 0.01414703, 0.06399614, 0.09812415, 0.07

In [ ]:
model.get_weights() # i pesi sono inizializzati ( da una Normal distribution... in modo particolare)

In [ ]:
model.summary()

In [ ]:
# Model summary: (10, 784) vuoldire: batch_size, dim dell'input (28x28 image)

In [ ]:
# ALternativa a Costruire il modello in modo artificioso:
model = keras.Sequential(
    [
        Flatten(input_shape=[28,28]),
        Dense(100, "sigmoid"),
        Dense(100, "sigmoid"),
        Dense(10, "softmax")
    ]
)
# input_shape può andare su any strato
# forse input_shape puà essere usato come assert anche negli strati successvi

### Compilazione
Cosa fa il compile?
Docs: Configures the model for training. https://www.tensorflow.org/api_docs/python/tf/keras/Model#compile

* optimizer: per il Gradient Descent, o anyway andare verso il minimo, trovare il modello più performante
* funzione di costo: per fare il gradiente, capire la direzione
* metrica per valutare il modello

In [ ]:
model.compile(
    optimizer=keras.optimizers.Adam(), # come oggetto o come stringa "adam"
    #loss = keras.losses.SparseCategoricalCrossentropy(from_logits = True),
    loss = keras.losses.SparseCategoricalCrossentropy(),
    metrics = [keras.metrics.SparseCategoricalAccuracy()] 
)
# tutte le metriche di aspettano y_true e y_pred (label e predizioni)
# la sparse categorical si aspetta le probabilità della classe predetta e non la classe (vedi docs)

In [ ]:
history = model.fit(
    x = x_train,
    y = y_train,
    batch_size = 32,
    epochs = 100,
    validation_split = 0.2,
    shuffle = True
)

In [ ]:
# Nota: keras permette di entrare nel dettaglio, a basso livello: fare strati custom, metriche custom, ...

In [ ]:
# Se faccio una fit dopo l'altra, la seconda continua dalla prima, senza sovrascrivere i pesi ( per farlo va ricreato il modello)

In [ ]:
history.history["loss"]
# dict: lista delle loss, poi delle accuracy, ... tutti i parametri

In [ ]:
# Accuracy sul Validation Set
import matplotlib.pyplot as plt

In [ ]:
plt.plot(history.history["val_sparse_categorical_accuracy"])

In [ ]:
plt.plot(history.history["loss"])
# nel mio caso non overfitta, non torna su la loss

### Callbacks
Usati per fare cose utili, come intervenire in diversi momenti dell'addestramento
+ ci sono Callback predefiniti: 
    * ex. early stoppping, 
    * checkpoint (salvare addestramento)
    * tensorboard

In [ ]:
# Early Stopping: ex. quando la loss smette di scendere, fermati
# I più usati dal prof. : CSVLogger, tensorboard, early stopping, model checkpoint

In [14]:
# Ricreo modello
model = keras.Sequential(
    [
        Flatten(input_shape=[28,28]),
        Dense(100, "sigmoid"),
        Dense(100, "sigmoid"),
        Dense(10, "softmax")
    ]
)
# Ricompilo
model.compile(
    optimizer=keras.optimizers.Adam(), # come oggetto o come stringa "adam"
    #loss = keras.losses.SparseCategoricalCrossentropy(from_logits = True),
    loss = keras.losses.SparseCategoricalCrossentropy(),
    metrics = [keras.metrics.SparseCategoricalAccuracy()] 
)

In [15]:
from keras.callbacks import CSVLogger, TensorBoard, EarlyStopping, ModelCheckpoint

In [16]:
import tensorflow as tf

In [17]:
# Per usare Tensorboard, creare cartella di log
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [18]:
tf.keras.callbacks.TensorBoard(
    log_dir='logdir',
    histogram_freq=0,
    write_graph=True,
    write_images=False,
    write_steps_per_second=False,
    update_freq='epoch',
    profile_batch=0,
    embeddings_freq=0,
    embeddings_metadata=None
)

In [24]:
%tensorboard --logdir logdir --host localhost --port 8088 #--path_prefix /tensorboard
# % per farlo partire in modalità notebook

ERROR: Failed to launch TensorBoard (exited with 1).
Contents of stderr:
2023-09-07 14:02:58.859891: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-07 14:02:58.951459: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-07 14:02:58.952095: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-07 14:03:00.252120: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/home/naska/miniconda3/lib/python3.9/site-packages/tensorboard_data_server/bin/server: /lib/x86_64-linux-gnu/libc.so.6: version `GLIBC_2.33' not found (required by /home/naska/miniconda3/lib/python3.9/site-packages/tens

In [23]:
callbacks = [
    TensorBoard(log_dir="logdir"),
    ModelCheckpoint("checkpoints/saved_model_{epoch:02d}.chkpt"), # see docs
    EarlyStopping(patience = 5, restore_best_weights=True) # https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/EarlyStopping
        # il parametro mode: significa: cosa vuoldire per quella metrica "migliorare"? verso l'alto o il basso (deve aumentare o diminuire?)
]

In [ ]:
history = model.fit(
    x = x_train,
    y = y_train,
    batch_size = 32,
    epochs = 100,
    validation_split = 0.2,
    shuffle = True,
    callbacks = callbacks
)

In [ ]:
# caricare modello
loaded_model = keras.models.load_model("/content/checkpoints/saved_model_16.chpkt")
# vedi nptebook del prof

### Tensorflow static vs dynamic

In [ ]:
import tensorflow as tf

In [ ]:
def sigmoid(t):
    print("Sono una sigmoide")
    return 1 / (1+tf.exp(-t))

In [ ]:
t = tf.constant([1.,2,3])

In [ ]:
sigmoid(t)

In [ ]:
# Decoratore per usare la mode statica
@tf.function
def sigmoid(t):
    print("Sono una sigmoide")
    return 1 / (1+tf.exp(-t))

In [ ]:
sigmoid(t)

In [ ]:
sigmoid(t)

In [ ]:
# nel secondo output manca la print.
# la modalitò statica è più veloce anche del 50%.

# la prima volta che la funz. viene chiamata TF fa un tracing, grafo computazionale statico delle operazioni che fa quella funzione.
# la seconda volta viene esguito il grafo e non il codice python. il tracing registra solo le operazioni di TF, la print è di python.

# Nota: argomento da approfondire, è complesso scrivere in static mode

In [ ]:
# Keras lavora in mode statica (fa un tracing)
# importante se faccio un layer custom, o altre cose custom

In [ ]:
# Keras può essere eseguito in mode dinamica
# cerca parametro run_eagerly (ex. nel metodo compile() di model

In [ ]:
sigmoid #  è un oggetto di TF

t1 = tf.constant([1.,2,3], [4.,5,6])
sigmoid(t1) # crea un altro grafo, fa un'alta volta il tracing
# diventa un wrapper di più funzioni "concrete", per due grafi comp. (uno per un tensore 1D come t, e uno per 2D come t2)

## Funzioni di Attivazione

In [ ]:
# vedi appunti

In [ ]:
# Confronto tra addestramenti con ReLU e con Sigmoide

In [ ]:
# vedi notebook del prof

### Parametro Training (layer e modelli)

In [28]:
from keras.layers import Dense, Dropout
import tensorflow as tf

In [26]:
dense_layer = Dense(10)

In [27]:
dropout_layer = Dropout(rate=0.5) # è proprio uno strato, posto dopo quello a cui si applica

In [33]:
t = tf.range(10, dtype=tf.float32)

In [34]:
t

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 2., 3., 4., 5., 6., 7., 8., 9.], dtype=float32)>

In [35]:
dropout_layer(t, training=True)

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([ 0.,  2.,  0.,  0.,  8., 10., 12.,  0., 16.,  0.], dtype=float32)>

In [ ]:
# Nota: in TF la classe model è sotto la classe layer (un model è un layer)

In [ ]:
from keras.layers import BatchNormalization


In [1]:
model = keras.Sequential(
    [
        Flatten(input_shape=(28, 28)),
        Dense(100, "relu"),
        BatchNormalization(),
        Dense(100, "relu"),
        BatchNormalization(),
        Dense(10, "softmax")
    ]
)

NameError: name 'keras' is not defined

In [2]:
model.summary()
#  solo la metù dei parametri degli strati di BatchNormalization è addestrabile

NameError: name 'model' is not defined

## Esercizio

In [1]:
# Vedi notebook "Esercizio"